In [1]:
#System path 등록
import sys
sys.path.insert(0, '../../tensorflow_keras_example')


In [2]:
from datasetslib.text8 import Text8
text8 = Text8()
# downloads data, converts words to ids, converts files to a list of ids
text8.load_data(clip_at=5000)

/Users/kimhyungrak/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kimhyungrak/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kimhyungrak/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kimhyungrak/anaconda3/lib/python3.7/site-packages/tensorf

Already exists: ./datasets/text8/text8.zip


(array([   8,  497,    7, ..., 1336,    2, 1066]), None, None)

In [3]:
#test_data 생성
import numpy as np
n_x = 5
n_y = 1
random_data5 = np.random.choice(n_x*100, n_x, replace=False).copy()
[text8.id2word[data_idx] for data_idx in random_data5]


['contemporary', 'set', 'advocates', 'n', 'male']

In [5]:
#model variable setup
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
batch_size = 128
hidden_size = 128
total_result = text8.vocab_len
learning_rate = 0.001

x_in = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, n_x, 1])
y_in = tf.compat.v1.placeholder(dtype=tf.float32, shape=[batch_size, total_result])

x_input_data = tf.unstack(x_in, axis=1)

In [6]:
#LSTM model structure define 

network_mode = 'multi'
use_dropout = False
number_layer =  2
keep_prob_ = 0.1
if network_mode == 'multi':    
    def create_rnn_cell():
        cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_units = hidden_size,
                                            state_is_tuple = True)
        return cell
    #drop out 추가
    if use_dropout:
        drops = [tf.compat.v1.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob_) 
                 for lstm in [create_rnn_cell()for _ in range(number_layer)]]
    else:
        drops = [create_rnn_cell()for _ in range(number_layer)]
    #Multi cell 추가 
    multi_cells = tf.compat.v1.nn.rnn_cell.MultiRNNCell(drops, state_is_tuple=True)
    
    lstm_out, last_state = tf.compat.v1.nn.dynamic_rnn(multi_cells, x_in, dtype=tf.float32)
    lstm_out_data = tf.unstack(lstm_out, axis=1)[-1]
else:
    lstm_cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(num_units=hidden_size)
    lstm_out, last_state = tf.compat.v1.nn.static_rnn(lstm_cell, x_input_data, dtype=tf.float32)
    lstm_out_data = lstm_out[-1]
lstm_out

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.


<tf.Tensor 'rnn/transpose_1:0' shape=(128, 5, 128) dtype=float32>

In [7]:

#logit calculation and define loss and optimizer
w = tf.compat.v1.get_variable(dtype=tf.float32, 
                              shape=[hidden_size, total_result], 
                              initializer=tf.random_normal_initializer(), 
                              name='input_weight')

b = tf.compat.v1.get_variable(dtype=tf.float32, shape=[total_result], name='input_bias')

logits =  tf.matmul(lstm_out_data, w) + b
loss = tf.compat.v1.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_in))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
# accuracy code 

accuracy_score = tf.compat.v1.reduce_mean(
    tf.cast(tf.equal(tf.argmax(logits,axis=1), tf.argmax(y_in, axis=1)), dtype=tf.float32))

In [ ]:
#excution
from datasetslib import util as dsu
from datasetslib import nputil

text8.reset_index()
n_batches = text8.n_batches_seq(n_tx=n_x, n_ty=n_y)

n_epochs = 1000
n_epochs_display=100

with tf.compat.v1.Session() as sess:
    tf.compat.v1.global_variables_initializer().run()
    
    for epoch in range(n_epochs):
        epoch_loss = 0
        epoch_accuracy = 0
        for i in range(n_batches):
            x_batch, y_batch = text8.next_batch_seq(n_tx=n_x,n_ty=n_y)
            y_batch = nputil.to2d(y_batch, unit_axis=1)
            y_onehot = np.zeros(shape=[batch_size, text8.vocab_len],dtype=np.float32)
            for i in range(batch_size):
                y_onehot[i, y_batch[i]] =1

            _, batch_accuracy, batch_loss = sess.run([optimizer, accuracy_score, loss], 
                                                     feed_dict={x_in: x_batch.reshape(-1, n_x, 1), y_in: y_onehot})
            epoch_loss += batch_loss
            epoch_accuracy += batch_accuracy
            
        if (epoch+1)%n_epochs_display==0:
            print(f'epoch:{n_epochs}........... epoch loss : {epoch_loss/n_batches} ...... epoch accuracy : {epoch_accuracy/n_batches}')
            
    

epoch:1000........... epoch loss : 0.46109799047311145 ...... epoch accuracy : 0.9309895833333334
epoch:1000........... epoch loss : 0.3340711084504922 ...... epoch accuracy : 0.9537760416666666
